In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Input
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_excel('/content/drive/MyDrive/Nhóm 7/Đồ án môn học/DATA_TRAINING/final_dataset_after_quality_control_V2.xlsx')
df.head()

,mssv,namsinh,gioitinh,dtb_toankhoa,dtb_tichluy,sotc_tichluy,diemtbhk_1,diemtbhk_2,diemtbhk_3,diemtbhk_4,diemtbhk_5,diemtbhk_6,diemtbhk_7,diemtbhk_8,diemtbhk_9,diemtbhk_10,diemtbhk_11,diemtbhk_12
0,0000AC05XPvAibaEXe9B2tolTZ0JLoBGbkQixQS6,13,0,7.91,7.91,62,7.79,8.28,7.56,0.00,5.9075,5.9075,5.9075,5.9075,5.9075,5.907500,5.907500,5.907500
1,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,13,0,8.79,8.79,129,8.84,9.00,9.11,8.75,8.5400,8.7400,8.4100,8.7700,8.7700,8.770000,8.770000,8.770000
2,00046394XPvAibaEXe+fmxcqgvribEcT4YmJhSFD,10,0,7.26,7.26,147,5.41,7.07,6.36,8.32,7.8600,8.0900,6.9200,7.8000,8.2000,7.336667,7.336667,7.336667
3,0006A0BBXPvAibaEXe/lMOwHQdw54DgUkWaqwb1u,13,0,7.27,7.27,26,7.27,7.27,7.27,7.27,7.2700,7.2700,7.2700,7.2700,7.2700,7.270000,7.270000,7.270000
4,000AD0D8XPvAibaEXe+RQyZpP6sq6qqIPZXybx3Q,15,0,8.08,8.08,136,8.85,8.64,8.41,8.05,7.4600,7.2800,8.5500,7.2400,8.0600,8.060000,8.060000,8.060000


In [ ]:
# df = df.drop(['khoa', 'hedt', 'khoahoc', 'diemtbhk_13', 'diemtbhk_14', 'diemtbhk_15', 'diemtbhk_16' , 'diemtbhk_17', 'diemtbhk_18', 'diemtbhk_19'], axis=1)
# df['gioitinh'] = df['gioitinh'].fillna('Nam')
# df['gioitinh'] = df['gioitinh'].replace({'Nam': 0, 'Nữ': 1})
# mean_nam_sinh = int(df['namsinh'].mean(skipna=True))
# df['namsinh'] = df['namsinh'].fillna(mean_nam_sinh)
# df['namsinh'] = df['namsinh'] - df['namsinh'].min()
# #df = df.fillna(0)
# df.head()

In [6]:

# Hàm xây dựng mô hình Phased LSTM
def build_phased_lstm(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(50, activation='tanh', recurrent_activation='sigmoid'),
        Dense(1)  # Đầu ra là một giá trị
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    return model

# Độ đo
def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100 if np.all(y_true != 0) else np.nan
    mpe = np.mean((y_true - y_pred) / y_true) * 100 if np.all(y_true != 0) else np.nan
    r2 = r2_score(y_true, y_pred)
    return {"R2 Score": r2, "MSE": mse, "RMSE": rmse, "MAE": mae, "MAPE": mape, "MPE": mpe}

# Cột ban đầu cho X
initial_columns = ['namsinh', 'gioitinh', 'dtb_toankhoa', 'dtb_tichluy', 'sotc_tichluy', 'diemtbhk_1']

# Lặp qua từng cột làm y
results = []
current_X_columns = initial_columns.copy()

for i in range(len(initial_columns), len(df.columns) - 1):  # Bắt đầu từ cột tiếp theo sau `initial_columns`
    target_column = df.columns[i]  # Cột y hiện tại
    next_target_column = df.columns[i + 1]  # Cột y mới (vòng tiếp theo)

    # Xác định X và y
    X = df[current_X_columns]
    y = df[target_column]

    # Chia dữ liệu
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    # Chuẩn hóa dữ liệu cho Phased LSTM
    X_train_lstm = np.expand_dims(X_train.to_numpy(), axis=-1)  # Thêm chiều
    X_val_lstm = np.expand_dims(X_val.to_numpy(), axis=-1)
    X_test_lstm = np.expand_dims(X_test.to_numpy(), axis=-1)

    # Xây dựng và huấn luyện mô hình Phased LSTM
    lstm_model = build_phased_lstm(input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2]))
    lstm_model.fit(X_train_lstm, y_train, validation_data=(X_val_lstm, y_val), epochs=10, batch_size=32, verbose=1)

    # Dự đoán và tính toán độ đo
    y_pred = lstm_model.predict(X_test_lstm).flatten()
    metrics = calculate_metrics(y_test, y_pred)
    metrics["Target Column"] = target_column
    results.append(metrics)

    # Cập nhật X cho vòng tiếp theo
    current_X_columns.append(target_column)

# Tạo DataFrame kết quả
results_df = pd.DataFrame(results)

# Hiển thị kết quả
print(results_df)

# Lưu kết quả ra file CSV
results_df.to_csv("phased_lstm_results.csv", index=False)


Epoch 1/10
298/298 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 19.3717 - mae: 3.4067 - val_loss: 0.3559 - val_mae: 0.4540
Epoch 2/10
298/298 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.2381 - mae: 0.3548 - val_loss: 0.0751 - val_mae: 0.1860
Epoch 3/10
298/298 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0561 - mae: 0.1559 - val_loss: 0.0301 - val_mae: 0.0958
Epoch 4/10
298/298 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0253 - mae: 0.0859 - val_loss: 0.0196 - val_mae: 0.0748
Epoch 5/10
298/298 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0163 - mae: 0.0667 - val_loss: 0.0138 - val_mae: 0.0549
Epoch 6/10
298/298 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0087 - mae: 0.0475 - val_loss: 0.0105 - val_mae: 0.0566
Epoch 7/10
298/298 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0053 - mae: 0.0386 - val_loss: 0.0065 - val_mae: 0.0345
Epoch 8/10
298/298 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0045 - mae: 0.0296 - val_loss: 0.0041 - val_mae: 0.0197
Epoch 9/10
298/298 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - l

In [7]:
print(results_df['R2 Score'].mean())
print(results_df['MSE'].mean())
print(results_df['RMSE'].mean())
print(results_df['MAE'].mean())

0.7971064903260988
0.8670447348395581
0.8793018296378782
0.46490261720803494
